In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name= "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.3.6: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    NVIDIA GeForce RTX 4070. Num GPUs = 1. Max memory: 11.719 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [30]:
from unsloth.chat_templates import get_chat_template
from unsloth.chat_templates import standardize_sharegpt

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

In [31]:
import os
with open(os.path.join(os.curdir, "Resources/Data/flanagan_basics.txt")) as f:
    dataset = eval(f.read())

In [33]:
with open(os.path.join(os.curdir, "Resources/Data/flanagan_motion_phases.txt")) as f:
    dataset1 = eval(f.read())

In [34]:
ds = dataset + dataset1

In [36]:
len(ds)

15

In [39]:
format_dataset = [{"conversations": conv} for conv in ds]
format_dataset[14]

{'conversations': [{'from': 'human', 'value': 'Return the object properties.'},
  {'from': 'gpt',
   'value': '(properties (size "medium") (texture "smooth") (color "red"))'},
  {'from': 'human', 'value': 'Return the tool properties.'},
  {'from': 'gpt',
   'value': '(properties (sharpness "high") (size "medium") (material "steel") (weight "light") (color "black") (edge "smooth"))'},
  {'from': 'human', 'value': 'Return the location properties.'},
  {'from': 'gpt',
   'value': '(properties (material "wood") (height 0.9) (accessibility "high") (surface-type "stable"))'},
  {'from': 'human', 'value': 'Return the goal details.'},
  {'from': 'gpt',
   'value': '(details (type "slice") (size "uniform") (quantity "multiple"))'},
  {'from': 'human', 'value': 'Return the task constraints.'},
  {'from': 'gpt',
   'value': '(task-constraints (cut-type "slices") (cut-uniformly "true") (remove-core "none") (remove-seeds "none") (maintain-shape-integrity "true"))'},
  {'from': 'human', 'value': 'Re

In [40]:
len(ds)

15

In [41]:
from datasets import Dataset
hugging_dataset = Dataset.from_list(format_dataset)
print(type(hugging_dataset))
print(hugging_dataset[0])

<class 'datasets.arrow_dataset.Dataset'>
{'conversations': [{'from': 'human', 'value': 'Can you explain Flanagan motion phases?'}, {'from': 'gpt', 'value': 'Sure! Flanagan motion phases describe the stages of movement in biomechanical analysis.'}, {'from': 'human', 'value': 'How many phases are there?'}, {'from': 'gpt', 'value': 'There are typically three main phases: preparation, execution, and follow-through.'}]}


In [42]:
std_dataset = standardize_sharegpt(hugging_dataset)
data = std_dataset.map(formatting_prompts_func, batched=True)

Standardizing format:   0%|          | 0/15 [00:00<?, ? examples/s]

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [43]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = data,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: We found double BOS tokens - we shall remove one automatically.


Tokenizing to ["text"] (num_proc=2):   0%|          | 0/15 [00:00<?, ? examples/s]

In [44]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

num_proc must be <= 15. Reducing num_proc to 15 for dataset of size 15.


Map (num_proc=15):   0%|          | 0/15 [00:00<?, ? examples/s]

In [45]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat about the initiation phase?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nIn this phase, the knife, food, and hand positions are set. The initial grip force and joint movements begin, gradually ramping up velocity to start the motion.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nSo it’s like positioning the knife and getting ready to cut?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nYes! Your brain and muscles prepare for controlled movement, just like a robot would.<|eot_id|>'

In [46]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                        \n\nIn this phase, the knife, food, and hand positions are set. The initial grip force and joint movements begin, gradually ramping up velocity to start the motion.<|eot_id|>                     \n\nYes! Your brain and muscles prepare for controlled movement, just like a robot would.<|eot_id|>'

In [47]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 15 | Num Epochs = 30 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/1,865,526,272 (1.30% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.999500
2,1.649500
3,1.713500
4,1.806700
5,2.197100
6,1.531800
7,1.903900
8,1.435700
9,1.682900
10,1.355500


## Inference

In [56]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

FastLanguageModel.for_inference(model)

messages = [
    {"role" : "user", "content" : "what is framenet for the task - cut the apple with knife, return it in lisp format"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

In [57]:
outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True, temperature = 1.5, min_p = 0.1)

In [58]:
text = tokenizer.batch_decode(outputs, skip_special_tokens = True)

In [59]:
print(text)

['system\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nuser\n\nwhat is framenet for the task - cut the apple with knife, return it in lisp formatassistant\n\n(fixed-action-model)\n(action (cut-apple (knife) (apple)) (result (pieces) (size) (quantity)) (outcome (successful-cut true) (error-type none) (error-description none)))']
